

---



---

---


**Перед запуском кода загрузите файлы, из которых собираем БД формат файлов .txt**





---





In [2]:
### Установка фреймворков
!pip install -U langchain # общий фреймворк
!pip install langchain_community # для работы с загрузчиком документов
!pip install -U sentence-transformers # для
!pip install -U langchain-chroma # для создание БД
!pip install -U deep-translator # переводчик (здесь используем Google Translator)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 351.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/6

In [5]:
# from transformers import pipeline

# # # Загрузка модели генерации текста (LLM) с Hugging Face
# # llm_model_name = "gpt2"  # Можно заменить на другую модель, например, более продвинутую
# # generator = pipeline("text-generation", model=llm_model_name)



"""Переводчик вопроса (запроса)"""
from deep_translator import GoogleTranslator

def query_translation(query) -> str:

  translated_query = GoogleTranslator(source='ru', target='en').translate(query)

  return translated_query


"""Переводчик ответа от LLM (генератора)"""
def response_translation(response) -> str:

  translated_response = GoogleTranslator(source='en', target='ru').translate(response)

  return translated_response


"""Переводчик файлов для Базы знаний"""
# (используется разово для импорта новых файлов в Базу знаний)
def text_translation(path_new_knowledge_file) -> str:

  translated_file = GoogleTranslator(source='ru', target='en').translate_file(path_new_knowledge_file)

  return translated_file # Вместо translated_file сделать импорт переведенных файлов в Базу Знаний


"""Тест работоспособности переводчика"""
# Пример запроса на русском языке
query = "Как работает система RAG?"

# Обработка запроса с переводом
response = query_translation(query)

# Вывод результата
print(response)






How does the RAG system work?


In [ ]:
# Загрузчик документов
from langchain.document_loaders import DirectoryLoader, TextLoader

# в loader указываем путь в папку с базой знаний
# path - укажите путь к ПАПКЕ с файлами для Базы знаний ()
# glob - файлы какого формата будут учитываться, тут ".txt" формат
# с другими форматами не тестил

loader = DirectoryLoader(path='/content', glob = './*.txt', loader_cls= TextLoader)
documents = loader.load()


#  Ссылка на сайт для подробного понимания DirectoryLoader (зажми 'Ctrl' и нажми на ссылку)
#  https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.directory.DirectoryLoader.html

In [ ]:
# Разбинеие на чанки
from langchain.text_splitter import RecursiveCharacterTextSplitter

# chunk_size - регулирует размер чанков
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200)
documents = text_splitter.split_documents(documents)

In [ ]:
# Эмбеддинг
from langchain.embeddings import HuggingFaceBgeEmbeddings

# Берём модель для эмбенига с Hugging Face
model_name = 'BAAI/bge-small-en-v1.5'
encode_kwargs = {'normalize_embeddings': True}
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    # вместо 'cpu' можно указать 'cuda' для более быстрой работы
    encode_kwargs=encode_kwargs
)

In [ ]:
# Создание Базы знаний через Chroma (без сохранения Базы знаний)
from langchain.vectorstores import Chroma
vectordb = Chroma.from_documents(documents, bge_embeddings)


"""
Добавить загрузку документов формата .txt
и парсинг информативных сайтов для обновления Базы Знаний(обновление скорее всего будет в настоящем времени)
Под обновлением подразумевается доступ к сайтам по URL ссылки
"""



In [ ]:
#  Ретривер
# as_retriever извлекает из векторной БД чанки
retriever = vectordb.as_retriever()

# response имеет вид списка, у него буквально тип данных 'list'
response = retriever.invoke("where is human?")


"""Добавить сюда переводчик для перевода вопроса с русского на английский"""


# используем response[0], потому что response - список,
#  а к списку не может быть применён метод page_content
# метод page_content выводит ответ в человечном виде
# о да, обязательно попробуй запустить без page_content)


"""Добавить сюда переводчик для перевода ответа с русского на английский"""


print(response[0].page_content)

In [ ]:
# И да доп. инфу и подробности можно искать через документацию langchain
# мужики, там есть поиск в правой верхнем углу (на сайте langchain)
# Вот ссылка (зажми 'Ctrl' и нажми на ссылку)
# https://python.langchain.com/docs/introduction/
